In [15]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

from typing import Union
from scipy.stats import t as StudentT

In [20]:
raw_data: pd.DataFrame = yf.download(tickers=['TCS.NS'], period='1y', interval='1h')
df: pd.DataFrame = raw_data.copy(deep=True)
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-09-16 09:15:00+05:30,3078.250000,3093.699951,3034.899902,3038.000000,3038.000000,1268611
2022-09-16 10:15:00+05:30,3038.000000,3042.350098,3030.199951,3031.550049,3031.550049,709122
2022-09-16 11:15:00+05:30,3031.550049,3031.899902,3017.000000,3023.600098,3023.600098,524052
2022-09-16 12:15:00+05:30,3023.149902,3033.550049,3023.149902,3024.649902,3024.649902,250850
2022-09-16 13:15:00+05:30,3024.649902,3031.000000,3018.000000,3019.550049,3019.550049,441518
...,...,...,...,...,...,...
2023-09-15 11:15:00+05:30,3578.250000,3590.000000,3576.050049,3590.000000,3590.000000,191283
2023-09-15 12:15:00+05:30,3589.899902,3603.000000,3587.850098,3596.600098,3596.600098,326499
2023-09-15 13:15:00+05:30,3597.100098,3599.000000,3581.149902,3589.250000,3589.250000,365607


Testing the basic implementation that generates the MonteCarlo series in pure python & no parallelization

In [19]:
class MonteCarloSimulation:
    #performs the monte carlo simulations
    @staticmethod
    def __simulate(df : pd.DataFrame, candles : int, iterations : int) -> tuple[pd.DataFrame, float, float]:
        #drift_calculations
        mu = df['log_returns'].mean()
        var = df['log_returns'].var()
        stdev = df['log_returns'].std()
        drift = mu - (0.5 * var * (df.shape[0] - 2) / df.shape[0])

        #daily returns calculations
        Z = StudentT.ppf(np.random.rand(candles, iterations), df = df.shape[0] - 1)
        daily_returns : np.ndarray = np.exp(np.array([drift]) + np.array([stdev]) * Z)

        #main simulation code
        price_paths = np.zeros_like(daily_returns)
        price_paths[0] = df['Close'].iloc[-1]
        
        for t in range(1, candles):
            price_paths[t] = price_paths[t - 1] * daily_returns[t]
        
        expected_value = pd.DataFrame(price_paths).iloc[-1].mean()
        expected_returns = 100 * (
        pd.DataFrame(price_paths).iloc[-1].mean() - price_paths[0,1]
        ) / pd.DataFrame(price_paths).iloc[-1].mean()
        
        price_paths = pd.DataFrame(price_paths)
        return price_paths, expected_value, expected_returns
    
    @staticmethod
    def __probability_calculator(price_paths : pd.DataFrame, higher_than : float) -> float:
        price_path_0 = price_paths.iloc[0,0]
        predicted = price_paths.iloc[-1]
        pred_list = list(predicted)
        if higher_than >= 0:
            over = [(i * 100) / price_path_0 for i in pred_list if ((i / price_path_0) - 1) >= higher_than]
        else:
            over = [(i * 100) / price_path_0 for i in pred_list if ((i / price_path_0) - 1) < higher_than]
        return (len(over) / price_paths.shape[1])

    #probability that we will get returns over a certain amount
    @staticmethod
    def results(
        prices : pd.Series, 
        candles : int = 20, 
        iterations : int = 10000, 
        higher_than : list = [0.0],
        return_value : str = 'all') -> float, dict None:
        
        #preprocessing
        df = pd.DataFrame()
        df['Close'] = prices.copy(deep = True)
        df['log_returns'] = np.log(1 + df['Close'].pct_change())

        #Perform Monte Carlo Simulations
        price_paths, expected_value, expected_returns = MonteCarloSimulation.__simulate(df, candles, iterations)
        
        res = {
        'Expected Value':expected_value,
        'Expected Returns':expected_returns,
        }

        for val in higher_than:
            res[val] = MonteCarloSimulation.__probability_calculator(price_paths, val)
        if return_value == 'probability':
            return MonteCarloSimulation.__probability_calculator(price_paths, higher_than[0])
        elif return_value == 'expected_value':
            return expected_value
        elif return_value == 'expected_returns':
            return expected_returns
        elif return_value == 'all':
            return res
        else:
            return None

In [ ]:
class MonteCarloSimple:
    __data: pd.Series
    __simulations: int
    __forecasts: int

    def __init__(self, data: pd.Series, simulations: int, forecasts: int) -> None:
        self.__data = data
        self.__simulations = simulations
        self.__forecasts = forecasts

    def __simulate(self) -> None:
        #drift_calculations
        log_returns: np.ndarray = np.log(1 + self.__data.pct_change())
        mu = log_returns.mean()
        var = log_returns.var()
        stdev = log_returns.std()
        drift = mu - (0.5 * var * (log_returns.shape[0] - 2) / log_returns.shape[0])

        #daily returns calculations
        Z = StudentT.ppf(np.random.rand(self.__forecasts, self.__simulations), df = log_returns.shape[0] - 1)
        daily_returns : np.ndarray = np.exp(np.array([drift]) + np.array([stdev]) * Z)

        #main simulation code
        price_paths = np.zeros_like(daily_returns)
        price_paths[0] = self.__data[-1]